In [1]:
import timeit
import pathlib
import os

import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

from models.architectures.multi import getModel


IMG_WIDTH =128
IMG_HEIGHT = 128
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64

data_dir = pathlib.Path(".\\data\\flower_photos")
image_count = len(list(data_dir.glob('*\\*.jpg')))
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])


list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))

def get_label(file_path):
  parts = tf.strings.split(file_path, os.path.sep)
  return parts[-2] == CLASS_NAMES

def get_metadata(file_path):
  parts = tf.strings.split(file_path, os.path.sep)
#   age = tf.strings.split(parts[-1],"_")[-2]
#   sex = tf.strings.split(parts[-1],"_")[-1]
  age = 69
  sex = 1
  return tf.constant([age, sex])

def decode_img(img):
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])


def process_path(file_path):
  label = get_label(file_path)
  metadata = get_metadata(file_path)
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  data = {"age_gen": metadata, "img": img}
  return data, label




def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()
  ds = ds.shuffle(buffer_size=shuffle_buffer_size)
  ds = ds.repeat()
  ds = ds.batch(BATCH_SIZE)
  ds = ds.prefetch(buffer_size=AUTOTUNE)

  return ds


labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
train_ds = prepare_for_training(labeled_ds)

# image_batch, label_batch = next(iter(train_ds))

# print(image_batch["age_gen"].numpy()[0])
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
]

model = getModel()
# tf.keras.utils.plot_model(model, 'multi_input_and_output_model.png', show_shapes=True)


print("compiling model....")
model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=METRICS
                  )
print("compilation completed....")


tensorboard_cbk = tf.keras.callbacks.TensorBoard(log_dir='./logs')
x,y = next(iter(train_ds.take(1)))
# print(y.numpy())
print(y.numpy().shape)

print("Training started....")

start = timeit.default_timer()
# model.fit(train_ds,
#           epochs=2,
#           steps_per_epoch=20,
#           )
stop = timeit.default_timer()


# TEST_LOSS, TEST_ACCURACY = model.evaluate(test_dataset, callbacks=[tensorboard_cbk])


# print("------------------------------------------------->")
# print("TEST LOSS: " + str(TEST_LOSS))
# print("TEST ACCURACY: " + str(TEST_ACCURACY))
# print('Time Taken to train: ', str((stop - start)/60) + " min")
# print("------------------------------------------------->")

# model.predict(train_ds.take(1))

# logits = model.predict(train_ds.take(1))
# print(logits[0])

compiling model....
compilation completed....
(64, 5)
Training started....


In [5]:
x = next(iter(list_ds.take(1)))

In [6]:
x.numpy()

b'data\\flower_photos\\dandelion\\14185089716_2a48298d17.jpg'

In [11]:
parts = tf.strings.split(x, os.path.sep)
age = tf.strings.split(parts[-1],"_")[-2]
sex = tf.strings.split(parts[-1],"_")[-1].numpy()[:-4]

In [14]:
int(age.numpy())

14185089716

In [15]:
int(sex)

ValueError: invalid literal for int() with base 10: b'2a48298d17'